## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/cities.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Severo-Kurilsk,RU,50.6789,156.1250,45.99,85,100,5.28,overcast clouds
1,1,Mataura,NZ,-46.1927,168.8643,43.48,86,48,5.21,scattered clouds
2,2,Faanui,PF,-16.4833,-151.7500,78.84,74,38,5.44,moderate rain
3,3,Albany,US,42.6001,-73.9662,67.39,71,27,12.33,scattered clouds
4,4,Pahrump,US,36.2083,-115.9839,103.24,10,98,10.00,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Let's find your perfect vacation!  What is the minimum temperature you are looking for (F)? "))
max_temp = float(input(" And the maximum temperature (F)? "))

Let's find your perfect vacation!  What is the minimum temperature you are looking for (F)? 80
 And the maximum temperature (F)? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

new_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)] 
new_df


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
7,7,Kapaa,US,22.0752,-159.3190,82.47,69,20,10.36,few clouds
13,13,Umm Kaddadah,SD,13.6017,26.6876,83.98,48,41,7.90,scattered clouds
20,20,Bathsheba,BB,13.2167,-59.5167,83.01,78,20,17.27,few clouds
31,31,Thinadhoo,MV,0.5333,72.9333,82.98,70,98,9.19,overcast clouds
47,47,Ormara,PK,25.2088,64.6357,81.41,91,25,11.92,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
669,669,Namatanai,PG,-3.6667,152.4333,82.74,68,13,3.74,few clouds
671,671,Saint-Georges,GD,12.0564,-61.7485,82.08,78,75,17.27,broken clouds
685,685,Dao,PH,11.3937,122.6856,82.02,85,40,1.57,scattered clouds
687,687,Mucurapo,TT,10.6667,-61.5333,81.97,82,61,9.75,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
nan_rows  = new_df.loc[new_df.isna().any(axis=1)]
nan_rows



,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = new_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
7,Kapaa,US,82.47,few clouds,22.0752,-159.3190,
13,Umm Kaddadah,SD,83.98,scattered clouds,13.6017,26.6876,
20,Bathsheba,BB,83.01,few clouds,13.2167,-59.5167,
31,Thinadhoo,MV,82.98,overcast clouds,0.5333,72.9333,
47,Ormara,PK,81.41,scattered clouds,25.2088,64.6357,
50,Butaritari,KI,82.47,broken clouds,3.0707,172.7902,
57,Concepcion Del Oro,MX,81.19,few clouds,24.6333,-101.4167,
69,Cayenne,GF,82.09,broken clouds,4.9333,-52.3333,
70,Hilo,US,83.28,haze,19.7297,-155.0900,
74,Victoria,HK,80.65,overcast clouds,22.2855,114.1577,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # change location
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    print(f"Retrieving Results for {index}: {row['City']}.")
    hotels = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = hotels['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for 7: Kapaa.
Closest hotel is Sheraton Kauai Coconut Beach Resort.
------------
Retrieving Results for 13: Umm Kaddadah.
Missing field/result... skipping.
------------
Retrieving Results for 20: Bathsheba.
Closest hotel is Atlantis Hotel.
------------
Retrieving Results for 31: Thinadhoo.
Missing field/result... skipping.
------------
Retrieving Results for 47: Ormara.
Closest hotel is Kenara Colony House for Rent.
------------
Retrieving Results for 50: Butaritari.
Closest hotel is Isles Sunset Lodge.
------------
Retrieving Results for 57: Concepcion Del Oro.
Closest hotel is Hotel Hacienda San Javier.
------------
Retrieving Results for 69: Cayenne.
Closest hotel is Hôtel Le Dronmi.
------------
Retrieving Results for 70: Hilo.
Closest hotel is Hilo Hawaiian Hotel.
------------
Retrieving Results for 74: Victoria.
Closest hotel is Mini Central.
------------
Retrieving Results for 76: Bahia Honda.
Closest hotel is Casa Gualberto - Deysi.
------------
Retrieving Re

In [9]:
hotel_df.head(20)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
7,Kapaa,US,82.47,few clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
13,Umm Kaddadah,SD,83.98,scattered clouds,13.6017,26.6876,
20,Bathsheba,BB,83.01,few clouds,13.2167,-59.5167,Atlantis Hotel
31,Thinadhoo,MV,82.98,overcast clouds,0.5333,72.9333,
47,Ormara,PK,81.41,scattered clouds,25.2088,64.6357,Kenara Colony House for Rent
50,Butaritari,KI,82.47,broken clouds,3.0707,172.7902,Isles Sunset Lodge
57,Concepcion Del Oro,MX,81.19,few clouds,24.6333,-101.4167,Hotel Hacienda San Javier
69,Cayenne,GF,82.09,broken clouds,4.9333,-52.3333,Hôtel Le Dronmi
70,Hilo,US,83.28,haze,19.7297,-155.0900,Hilo Hawaiian Hotel
74,Victoria,HK,80.65,overcast clouds,22.2855,114.1577,Mini Central


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [11]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig
#savefig = "WeatherPy_vacation_map.png"


Figure(layout=FigureLayout(height='420px'))